#Context

AirPay is Running A Cashback Campaign For All Users Who Make Qr Payment At Our Acquired Merchants. Details: Cashback 30% Of Transaction Value, Maximum 20k. Some Users Are Cooperating With Merchant To Abuse This Promotion. We Have Monitored Their Transactions For A Couple Of Days. Apply Your Own Defined Logic To Find Answer For Following Questions.
* Which Gmv Range Has The Most Transaction?
* List The 3 Shop Owner UIDs Who Are Most Likely Fraud? Explain Your Answer
* List The 3 User IDs Who Are Most Likely Fraud? Explain Your Answer


# Library import
We import all the required Python libraries

In [681]:
# Data manipulation
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta

# Options for pandas
from IPython.core.display import display, HTML
pd.options.display.max_columns = None
display(HTML("<style>.container { width:85% !important; }</style>"))
pd.options.display.float_format = '{:,}'.format

# Visualizations
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# ML
from sklearn.datasets.samples_generator import (make_blobs,
                                                make_circles,
                                                make_moons)
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import decomposition
from sklearn import datasets

##Helper functions

In [0]:
# Styling
def highlight_cols(s):
    color = 'red'
    return 'background-color: %s' % color

def short_timedelta_highlight(s):    
  '''highlight the timedelta is less than 15 mins yellow.'''
  is_short = s<timedelta(minutes=30)
  return ['background-color: yellow' if v else '' for v in is_short]

In [0]:
def time_variance(sorted_time_column):
  '''This function takes a sorted column of datetime values, and return the minimum timedelta among them'''
  if sorted_time_column.shape[0] < 3:
    return None
  else:
    time_column_A = sorted_time_column[:-1].reset_index(drop=True)
    time_column_B = sorted_time_column[1:].reset_index(drop=True)
    time_diff = time_column_A - time_column_B
    return time_diff.min()

# Solution

In [684]:
from google.colab import drive
drive.mount('/content/gdrive')

Orders = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/BI Test 1/BI-Test-Part-3-Logical-Test-1.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Cleansing & EDA

In [685]:
Orders.head()

,txn_time,txn_date,order_id,uid,shop_id,shop_owner_uid,gmv,rebate
0,10/30/2019 13:37,10/30/2019,142008725,102161515,1005,100054430,70000,20000
1,10/30/2019 20:26,10/30/2019,142119377,102867263,10050,1792841,80000,20000
2,10/30/2019 12:22,10/30/2019,141986418,100401383,10075,100155720,125000,20000
3,10/30/2019 9:46,10/30/2019,141941958,103213155,10088,103894677,82000,20000
4,10/30/2019 21:56,10/30/2019,142139275,101003290,10142,103909421,260000,20000


In [686]:
Orders.describe(include='all')

,txn_time,txn_date,order_id,uid,shop_id,shop_owner_uid,gmv,rebate
count,4611,4611,"4,611.0","4,611.0","4,611.0","4,611.0","4,611.0","4,611.0"
unique,2521,4,nan,nan,nan,nan,nan,nan
top,11/2/2019 13:05,11/2/2019,nan,nan,nan,nan,nan,nan
freq,8,1529,nan,nan,nan,nan,nan,nan
mean,NaN,NaN,"142,459,452.62806332","89,407,785.15463023","11,012.236391238343","82,374,120.97397527","138,985.27586206896","16,813.508349598786"
std,NaN,NaN,"282,885.29053772055","33,332,276.69120953","9,006.849780494942","40,535,045.39838986","193,990.40874467144","6,520.511497350525"
min,NaN,NaN,"141,901,047.0","1,002,600.0",136.0,"1,000,025.0",1.0,0.0
25%,NaN,NaN,"142,232,061.0","100,441,381.5","5,643.0","100,305,004.0","70,000.0","19,500.0"
50%,NaN,NaN,"142,485,396.0","101,495,141.0","8,433.0","102,817,559.0","84,926.0","20,000.0"
75%,NaN,NaN,"142,723,326.5","103,266,260.0","10,873.0","103,489,596.0","139,000.0","20,000.0"


In [0]:
Orders.txn_time = pd.to_datetime(Orders.txn_time)

In [688]:
px.histogram(Orders, x='gmv')

##Fraud detection

Fraud indicators:
1. An user makes an unreasonably high number of transactions
2. A pair of shop owner - user makes an unreasonably high number of transactions
3. The rebate is at maximum, i.e. 20k
4. The GMV is around the bare minimum so that the most number of transactions can be made, i.e. 20k/30% = 66k-70k
5. Transactions are frequent within a short period of time
6. Rebate sum by user is maximised

### Indicator 1: An user makes an unreasonably high number of transactions

In [689]:
# Order count by uid
df = pd.DataFrame(Orders.groupby('uid').agg(['sum', 'count']))['rebate']
df.columns = ['Rebate_sum_by_user','Order_count_by_user']
px.histogram(df, x='Order_count_by_user')

In [0]:
Orders_extra = pd.merge(Orders, df, left_on='uid', right_index=True, how = 'left')

In [0]:
indicator_1 = Orders_extra['Order_count_by_user']>=5

### Indicator 2: A pair of shop owner - user makes an unreasonably high number of transactions

In [692]:
# Order count by shop-user pair
df = pd.DataFrame(Orders.groupby(['uid', 'shop_id']).agg(['sum', 'count']))['rebate']
df.columns = ['Rebate_sum_by_shop&user','Order_count_by_shop&user']
px.histogram(df, x='Order_count_by_shop&user')

In [0]:
Orders_extra = pd.merge(Orders_extra, df, left_on=['uid', 'shop_id'], right_index=True)

In [0]:
indicator_2 = Orders_extra['Order_count_by_shop&user']>=3

### Indicator_3: The rebate is at maximum, i.e. near 20k

In [695]:
px.histogram(Orders_extra, x='rebate')

Since most rebate are 20k, i.e. most users take full advantage of the rebate, this indicator is not effective. We shall drop this indicator.

### Indicator_4: The GMV is minimum so that the most number of transactions can be made, i.e. 20k/30% = 66k-70k

In [696]:
px.histogram(Orders_extra, x='gmv')

In [0]:
indicator_4 = (Orders_extra['gmv']>60000)&(Orders_extra['gmv']<80000)

###Indicator_5: Transactions are frequent within a short period of time

In [698]:
Orders_extra.sort_values(by=['txn_time'], ascending=False, inplace=True)

df = Orders_extra.groupby(by='uid')[['txn_time', 'gmv', 'rebate', 'order_id']]              \
  .agg({'txn_time': time_variance, 'gmv': 'mean', 'rebate': 'mean', 'order_id': 'count'})   \
  .dropna()                                                                                 \
  .reset_index()                                                                            \
  .sort_values('txn_time')
df.columns = ['uid', 'time_variance', 'average gmv', 'average rebate', 'count of order']
df.head(10).style.apply(short_timedelta_highlight, subset=['time_variance'])

,uid,time_variance,average gmv,average rebate,count of order
51,100265478,0 days 00:00:00,100000.000000,20000.000000,3
117,101233974,0 days 00:00:00,80025.000000,20000.000000,4
118,101234059,0 days 00:00:00,80339.500000,20000.000000,4
119,101234453,0 days 00:00:00,87525.000000,20000.000000,4
120,101234995,0 days 00:00:00,90125.000000,20000.000000,4
245,103650095,0 days 00:00:00,94285.714286,20000.000000,7
121,101235042,0 days 00:00:00,81632.000000,20000.000000,4
122,101263534,0 days 00:00:00,80104.000000,20000.000000,4
204,103282624,0 days 00:00:00,55000.000000,11428.571429,7
124,101285361,0 days 00:00:00,45000.000000,10000.000000,4


In [699]:
fig = go.Figure()
trace = go.Histogram(x=df['time_variance'].astype('timedelta64[m]'),
                     xbins=dict(
                     start=0,
                     size=20),
                     autobinx=False
                     )
fig.add_trace(trace)

In [0]:
indicator_5_uid = df.loc[df.time_variance <= timedelta(minutes=20), 'uid']
indicator_5 = Orders_extra['uid'].isin(indicator_5_uid)

### Indicator 6: Rebate sum by user is maximised

In [701]:
px.histogram(Orders_extra,
             x = 'Rebate_sum_by_user')

In [0]:
indicator_6 = Orders_extra['Rebate_sum_by_user']==Orders_extra['Rebate_sum_by_user'].max()

### Combine all indicators

In [0]:
for indicator in ['indicator_1', 'indicator_2', 'indicator_4', 'indicator_5', 'indicator_6']:
  Orders_extra[indicator] = eval(indicator)

In [0]:
Orders_extra['Number of flags'] = Orders_extra.indicator_1*1 + Orders_extra.indicator_2*1 \
                                 + Orders_extra.indicator_4*1 + Orders_extra.indicator_5*1 \
                                 + Orders_extra.indicator_6*1

In [705]:
Suspected_orders = Orders_extra[Orders_extra['Number of flags']>=4]
print(Suspected_orders.shape[0])
Suspected_orders.style.applymap(highlight_cols, subset=['txn_time', 'uid', 'shop_id', 'Rebate_sum_by_user', 'Number of flags'])

53


,txn_time,txn_date,order_id,uid,shop_id,shop_owner_uid,gmv,rebate,Rebate_sum_by_user,Order_count_by_user,Rebate_sum_by_shop&user,Order_count_by_shop&user,indicator_1,indicator_2,indicator_4,indicator_5,indicator_6,Number of flags
3904,2019-11-02 19:39:00,11/2/2019,142832555,100702363,5643,103435279,70000,20000,160000,8,80000,4,True,True,True,True,False,4
3292,2019-11-02 19:36:00,11/2/2019,142832032,103086848,10979,102817559,70000,20000,80000,7,80000,7,True,True,True,True,False,4
4058,2019-11-02 18:20:00,11/2/2019,142816173,102469889,6732,1924682,70000,20000,118000,6,60000,3,True,True,True,True,False,4
4016,2019-11-02 18:19:00,11/2/2019,142816046,102469889,6659,101476221,70000,20000,118000,6,58000,3,True,True,True,True,False,4
3279,2019-11-02 18:17:00,11/2/2019,142815571,103514096,10979,102817559,70000,20000,80000,6,80000,6,True,True,True,True,False,4
4057,2019-11-02 18:17:00,11/2/2019,142815699,101166204,6732,1924682,70000,20000,120000,6,60000,3,True,True,True,True,False,4
4015,2019-11-02 18:17:00,11/2/2019,142815562,101166204,6659,101476221,70000,20000,120000,6,60000,3,True,True,True,True,False,4
3251,2019-11-02 12:40:00,11/2/2019,142735272,103282624,10979,102817559,70000,20000,80000,7,80000,7,True,True,True,True,False,4
3224,2019-11-02 09:00:00,11/2/2019,142686046,102696939,10979,102817559,70000,20000,60000,6,60000,6,True,True,True,True,False,4
3221,2019-11-02 08:58:00,11/2/2019,142685689,1191456,10979,102817559,70000,20000,80000,8,80000,8,True,True,True,True,False,4


In [706]:
Suspected_uid = Suspected_orders['uid'].unique().tolist()
Suspected_shop_id = Suspected_orders['shop_id'].unique().tolist()
print(f' {len(Suspected_uid)} Suspected uid: {Suspected_uid}')
print(f' {len(Suspected_shop_id)} Suspected uid: {Suspected_shop_id}')

 21 Suspected uid: [100702363, 103086848, 102469889, 103514096, 101166204, 103282624, 102696939, 1191456, 102849459, 100259747, 103366324, 103653392, 103667901, 103653018, 103248548, 1924682, 103023839, 100823558, 103005918, 103868446, 102144081]
 18 Suspected uid: [5643, 10979, 6732, 6659, 8576, 8591, 6917, 4809, 7181, 6909, 5956, 6840, 7536, 7653, 4342, 29905, 5808, 10507]


## Fraud detection using ML

In [718]:
Orders_extra.reset_index(inplace=True,drop=True)
Orders_extra.head()

,txn_time,txn_date,order_id,uid,shop_id,shop_owner_uid,gmv,rebate,Rebate_sum_by_user,Order_count_by_user,Rebate_sum_by_shop&user,Order_count_by_shop&user,indicator_1,indicator_2,indicator_4,indicator_5,indicator_6,Number of flags,PC1,PC2,PC3,Cluster
0,2019-11-02 23:58:00,11/2/2019,142871368,100985133,10824,103157062,286000,20000,40000,2,40000,2,False,False,False,False,False,0,0.5004698771993966,1.3032111365545611,-0.6011777216578948,0
1,2019-11-02 23:47:00,11/2/2019,142870551,103868150,1238,1621933,100000,20000,60000,3,60000,3,False,True,False,False,False,1,-0.37026272236821106,1.255187833923242,-0.9868146315192855,0
2,2019-11-02 23:38:00,11/2/2019,142869904,103901631,6072,103498278,135000,20000,60000,3,20000,1,False,False,False,True,False,1,-0.30824962998325545,1.3093273787746897,-0.8757358145899812,0
3,2019-11-02 23:37:00,11/2/2019,142869825,103901631,6030,103498440,135000,20000,60000,3,20000,1,False,False,False,True,False,1,-0.3086391418638792,1.3092780717336794,-0.8752879070258183,0
4,2019-11-02 23:35:00,11/2/2019,142869700,103901631,6050,103498999,135000,20000,60000,3,20000,1,False,False,False,True,False,1,-0.30855063293919327,1.309347648446965,-0.8741183297406262,0


In [0]:
# Choose relevant columns
object_cols = ['uid', 'shop_id']
number_cols = ['gmv' ,'Rebate_sum_by_user', 'rebate', 'Order_count_by_user']
date_cols = ['txn_time']

In [709]:
# Create training data df
df = Orders_extra[object_cols + number_cols + date_cols]
df['txn_time'] = (df.txn_time - pd.to_datetime('1970-01-01')).dt.total_seconds()
df[object_cols] = df[object_cols].astype(str)

# Onehotencode categorical data, drop na values
df = df.join(pd.get_dummies(df[object_cols]))
df.drop(object_cols,axis=1,inplace=True)
df.dropna(axis=0,inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
# Scale numerical columns
sc = StandardScaler()
numeric_cols = df.select_dtypes(['int64','float64']).columns

sc_transformed_cols = pd.DataFrame(sc.fit_transform(df[numeric_cols]), columns=numeric_cols, index = df.index)
df.drop(numeric_cols,axis=1,inplace=True)
df = df.merge(sc_transformed_cols,left_index= True, right_index = True, how='left')

In [0]:
# fit KMeans model
no_of_clusters = 3
model = KMeans(n_clusters=no_of_clusters, init='k-means++', max_iter=100, n_init=1)
cluster = pd.DataFrame(model.fit_predict(df))
cluster.columns = ['Cluster']

In [0]:
# extract 3 PCs using PCA
pca = decomposition.PCA(n_components=3)
pca.fit(df)
df_PCA = pd.DataFrame(pca.transform(df))
df_PCA.columns = ['PC1', 'PC2', 'PC3']

In [0]:
df_PCA = df_PCA.merge(cluster, left_index=True, right_index=True)

In [0]:
Orders_extra = Orders_extra.merge(df_PCA, left_index=True, right_index=True)

In [723]:
px.scatter_3d(df_PCA,
           x='PC1',
           y='PC2',
           z='PC3',
           color='Cluster')